In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

# from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load



In [ ]:
kG = 1000*u.cm**(-1/2)*u.g**(1/2)*u.s**(-1)
R_sun = const.R_sun.cgs
M_sun = const.M_sun.cgs
G = const.G.cgs
mdot = (1e-8*u.M_sun/u.yr).to(u.g/u.s)

print((kG**(4/7)*R_sun**(12/7)/(M_sun**(1/7)*(mdot)**(2/7)*(4*G)**(1/7))).to(u.au))
params = code_gas.Params()

def mdot_t(mdot):

    return 10**((1.32+np.log10(mdot/(u.M_sun/u.yr)))/(-1.07))*u.yr

t = mdot_t((1e-8*u.M_sun/u.yr)).to(u.Myr).value

# print(r_magnetic_cavity(t, params))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
linestyle = ["-", ":", "-."]
for i, (model, e_el, e_h) in enumerate(zip(['irradiated', 'Lambrechts_mixed', 'Lambrechts_mixed'], [1e-2,1e-2, 1], [0.5,0.5, 1])):
    params = code_gas.Params(H_r_model=model, epsilon_el=e_el, epsilon_heat=e_h, star_mass=1*u.M_sun.to(u.M_earth))
    t = 0.1
    r = np.geomspace(1e-2,1e2,100)
    M_0 = M0_pla(r, t, sigma_gas_steady_state(r, t, params), params)
    M_iso = M_peb_iso(r, t, params)
    ratio1 = v_peb_app(r, M_0, t, params)/v_crit_settling(r, M_0, t, params)
    # ratio2 = v_peb_app(r, M_iso, t, params)/v_crit_settling(r, M_iso, t, params)

    ax.plot(r, ratio1, color='gold',  label='v_peb/v_crit at M_0, '+str(model)+", "+str(e_el), linewidth=3, linestyle=linestyle[i])
    # ax.plot(r, ratio2, color='navy',  label='v_peb/v_crit at M_iso, '+str(model)+", "+str(e_el), linewidth=3, linestyle=linestyle[i])


ax.axhline(1, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$\Delta \rm v /v_{*}$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity for $M_0$', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)
ax.legend()
ax.grid(which='both', linestyle='--', linewidth=0.5, color='gray')

In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,300)
R = 0.1
params = code_gas.Params(H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1, star_mass=1*u.M_sun.to(u.M_earth))
ratio = v_peb_app(R,M, t, params)/v_crit_settling(R, M, t, params)
ratio2 = v_circ(R, M, t, params)/v_crit_settling(R, M, t, params)

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(M, ratio, color='black',  label='v_app/v_crit')
ax.plot(M, ratio2, color='red',  label='v_circ/v_crit')
ax.plot(M, v_hw(R,t,params)/v_crit_settling(R, M, t, params), color='blue', label='v_hw/v_crit')

ax.set_xlabel('$M [M_{\oplus}]$', fontsize=20)
ax.set_ylabel(r'$\Delta \rm v /v_{*}$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)
ax.legend()
ax.grid(which='both', linestyle='--', linewidth=0.5, color='gray')
ax.axhline(1, color='black', linestyle='--', linewidth=1, label='v_crit')


In [ ]:
# Create meshgrid for pos and mass
Pos, Mass = np.meshgrid(pos, mass, indexing='ij')

# Calculate ratio v_app / v_crit element-wise
ratio = v_app(Pos, Mass, t, params) / v_crit(Pos, Mass, t, params)

# Plot heatmap
plt.figure(figsize=(8, 6))
im = plt.imshow(ratio, 
                extent=[mass.min(), mass.max(), pos.min(), pos.max()],
                origin='lower', aspect='auto', 
                norm=plt.LogNorm(), cmap='viridis')

plt.colorbar(im, label='v_app / v_crit (ratio)')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('mass')
plt.ylabel('pos')
plt.title('Heatmap of ratio v_app / v_crit')


In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,300)

X,Y = np.meshgrid(r,M)
print("X", X)

print("Y", Y)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth))
ratio = v_peb_app(X, Y, t, params)/v_crit_settling(X, Y, t, params)

fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.viridis
lev_exp = np.arange(np.floor(np.log10(ratio.min())),
                    np.ceil(np.log10(ratio.max())+1), step=0.1)
levs = np.power(10, lev_exp)
contour = ax.contourf(
    X, Y, v_approach(X, Y, t, params)/v_crit_settling(X, Y, t, params), levels=levs, cmap=cmap, norm = colors.LogNorm(vmin=levs.min(), vmax=levs.max()))


# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[10], colors='grey', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[100], colors='black', linewidths=2.5, linestyles='solid')


# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$v_{\rm approach}/ v_{\rm sett, crit}$')
cbar.set_label(r'$v_{\rm approach}/ v_{\rm sett, crit}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)

ax.loglog(r, M_Bondi_Hill_trans(r, t, params), color = 'red', linestyle='--')
ax.loglog(r, M_3D_2DH_trans(r, t, params), color = 'orange', linestyle='--')
ax.loglog(r, M_3D_2DB_trans(r, t, params), color = 'green', linestyle='--')

#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,300)

X,Y = np.meshgrid(r,M)
print("X", X)

print("Y", Y)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth))
ratio = v_sh(X, Y, t, params)/v_crit_settling(X, Y, t, params)

fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.viridis
lev_exp = np.arange(np.floor(np.log10(ratio.min())),
                    np.ceil(np.log10(ratio.max())+1), step=0.1)
levs = np.power(10, lev_exp)
contour = ax.contourf(
    X, Y, ratio, levels=levs, cmap=cmap, norm = colors.LogNorm(vmin=levs.min(), vmax=levs.max()))


# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[10], colors='grey', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[100], colors='black', linewidths=2.5, linestyles='solid')


# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$v_{\rm approach}/ v_{\rm sett, crit}$')
cbar.set_label(r'$v_{\rm approach}/ v_{\rm sett, crit}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)

ax.loglog(r, M_Bondi_Hill_trans(r, t, params), color = 'red', linestyle='--')
ax.loglog(r, M_3D_2DH_trans(r, t, params), color = 'orange', linestyle='--')
ax.loglog(r, M_3D_2DB_trans(r, t, params), color = 'green', linestyle='--')

#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity shear limit', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,300)

X,Y = np.meshgrid(r,M)
print("X", X)

print("Y", Y)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth))
ratio = v_hw(X,  t, params)/v_crit_settling(X, Y, t, params)

fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.viridis
lev_exp = np.arange(np.floor(np.log10(ratio.min())),
                    np.ceil(np.log10(ratio.max())+1), step=0.1)
levs = np.power(10, lev_exp)
contour = ax.contourf(
    X, Y, ratio, levels=levs, cmap=cmap, norm = colors.LogNorm(vmin=levs.min(), vmax=levs.max()))


# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[10], colors='grey', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[100], colors='black', linewidths=2.5, linestyles='solid')


# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$v_{\rm approach}/ v_{\rm sett, crit}$')
cbar.set_label(r'$v_{\rm approach}/ v_{\rm sett, crit}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)

ax.loglog(r, M_Bondi_Hill_trans(r, t, params), color = 'red', linestyle='--')
ax.loglog(r, M_3D_2DH_trans(r, t, params), color = 'orange', linestyle='--')
ax.loglog(r, M_3D_2DB_trans(r, t, params), color = 'green', linestyle='--')

#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity headwind limit', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,100)

X,Y = np.meshgrid(r,M)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth))
ratio = np.exp(-0.5*(v_peb_app(X, Y, t, params)/v_crit_settling(X, Y, t, params))**2)
#ratio = v_peb_app(X,Y, t, params)/v_crit_settling(X, t, Y, params)
fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.viridis

contour = ax.contourf(
    X, Y, ratio, levels=50, cmap=cmap)

# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[10], colors='grey', linewidths=2.5, linestyles='solid')
c1 = ax.contour(X, Y, ratio, levels=[100], colors='black', linewidths=2.5, linestyles='solid')


# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$f_{\rm set}$')
cbar.set_label(r'$f_{\rm set}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)


#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'$f_{\rm set}$', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,100)

X,Y = np.meshgrid(r,M)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth), St_const=None)
#ratio = v_hw(X, t, params)/v_crit_settling(X, Y, t, params)
#ratio = v_sh(X, Y, t, params)/v_crit_settling(X, Y, t, params)

ratio = v_hw(X, t, params)/v_sh(X, Y, t, params)
fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.inferno
lev_exp = np.arange(np.floor(np.log10(ratio.min())),
                    np.ceil(np.log10(ratio.max())+1), step=0.1)
levs = np.power(10, lev_exp)
contour = ax.contourf(
    X, Y, ratio, levels=levs, cmap=cmap, norm = colors.LogNorm(vmin=levs.min(), vmax=levs.max()))

# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')



# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$v_{\rm approach}/ v_{\rm sett, crit}$')
cbar.set_label(r'$v_{\rm HW}/ v_{\rm SH}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)


#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
fig, axs = plt.subplots (1,1, figsize=(8, 6))
mass = 1e-6
r = np.geomspace(1e-2, 1e2)
t=0.1
params = code_gas.Params(H_r_model='irradiated',epsilon_el=1, epsilon_heat=1,  star_mass=1*u.M_sun.to(u.M_earth))
axs.loglog(r, (v_peb_app(r, mass, t, params)*u.au/u.Myr).to(u.cm/u.s)/v_k(r, params), label ="approach velocity")
axs.loglog(r, (v_crit_settling(r, mass, t, params)*u.au/u.Myr).to(u.cm/u.s)/v_k(r, params), label ="critical settling velocity")
# axs.loglog(r, st_frag_drift(r, t, params))
# axs.loglog(r, f_set(r, mass, t, params))
axs.legend()

In [ ]:
fig, axs = plt.subplots (1,1, figsize=(8, 6))
x = np.geomspace(1e-4, 10, 100)
axs.loglog(x, np.exp(-0.5*x**2))
axs.grid()

In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,200)

X,Y = np.meshgrid(r,M)
params = code_gas.Params(H_r_model='irradiated', star_mass=1*u.M_sun.to(u.M_earth))
f = f_set(X, Y, t, params)
fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.inferno

contour = ax.contourf(
    X, Y, f_set(X, Y, t, params), cmap=cmap)


ratio = v_peb_app(X,Y, t, params)/v_crit_settling(X, Y,t, params)
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=3, linestyles='solid')

# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label(r'$f_{\rm set}$', fontsize=20)


#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'settling fraction', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
np.exp(-1/2)

In [ ]:
fig, axs =plt.subplots(1,1, figsize=(8, 6))
M=1e-5
axs.loglog(r, f_set(r,M,t,params))
axs.grid(which='both', linestyle='--', linewidth=0.5)

In [ ]:
t = 0.1
r = np.geomspace(1e-2,1e2,100)
M = np.linspace(1e-7,1e3,100)

X,Y = np.meshgrid(r,M)
params = code_gas.Params(H_r_model='Lambrechts_mixed', star_mass=1*u.M_sun.to(u.M_earth))
ratio = v_peb_app(X,Y, t, params)/v_crit_settling(X, t, Y, params)
fig, ax = plt.subplots(figsize=(8, 6))
cmap = plt.cm.viridis
lev_exp = np.arange(np.floor(np.log10(ratio.min())),
                    np.ceil(np.log10(ratio.max())+1), step=0.1)
levs = np.power(10, lev_exp)
contour = ax.contourf(
    X, Y, ratio, levels=levs, cmap=cmap, norm = colors.LogNorm(vmin=levs.min(), vmax=levs.max()))

# Add a white contour line where ratio = 1
c1 = ax.contour(X, Y, ratio, levels=[1], colors='white', linewidths=2.5, linestyles='solid')



# Optionally, make the line thicker for better visibility
for c in c1.collections:
    c.set_linewidth(3)
    c.set_zorder(10)  # Draw on top
cbar = plt.colorbar(contour, ax=ax, label=r'$v_{\rm approach}/ v_{\rm sett, crit}$')
cbar.set_label(r'$v_{\rm approach}/ v_{\rm sett, crit}$', fontsize=20)
cbar.ax.axhline(1, color='white', linewidth=3)
cbar.ax.axhline(10, color='grey', linewidth=3)
cbar.ax.axhline(100, color='black', linewidth=3)

#cbar.ax.text(1.5, 1, '1', color='black', va='center', ha='left', clip_on=False, transform=cbar.ax.get_yaxis_transform())

ax.loglog(r, M0_pla_Mstar(r,t,sigma_gas_steady_state(r, t, params), params), color = 'lightblue', linestyle='--') 
ax.loglog(r, M_peb_iso(r,t, params), color = 'violet', linestyle='--') 

ax.set_xlabel('r [AU]', fontsize=20)
ax.set_ylabel(r'$M_{\rm p} [M_{\oplus}]$', fontsize=20)
ax.set_title(r'approach velocity/critical settling velocity', fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=15, length=10)
ax.tick_params(axis='both', which='minor', labelsize=16, length=6)
plot.all_y_ticks(ax, num_ticks=100)


In [ ]:
1/np.sqrt(2*np.pi)

In [ ]:
3/2*(4**(1/3))/(3**(1/3))

In [ ]:
params = code_gas.Params(H_r_model = 'irradiated')
sim_params = code_gas.SimulationParams()
r = np.geomspace(1e-1, 50, 100)
t = 0.3
r0 = 0.1
mstar = (np.linspace(0.1, 10, 100)*u.M_sun).to(u.M_earth).value

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color='gold', label = 'initial mass'+str(params.H_r_model))
axs[0].axhline(8.314566896275828e-06, color='black', linestyle='--')
axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color='gold', label = 'initial mass'+str(params.H_r_model))

for m in mstar:

    params = code_gas.Params(H_r_model = 'irradiated', star_mass = m, M_dot_star=0)
    print('star mass', params.star_mass*u.M_earth.to(u.M_sun))
    axs[1].scatter(m*u.M_earth.to(u.M_sun), M0_pla(r0, t, sigma_gas_steady_state(r0, t, params), params), color='gold', label =str(params.star_mass))

for i in range(2):
    axs[i].tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
    axs[i].tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
    axs[i].grid(True, which='both', linestyle='--', alpha=0.5)
    axs[i].set_title('Initial mass of embros', fontsize=16)

axs[0].set_xlabel(r'$r$ [AU]', fontsize=16)
axs[1].set_xlabel(r'$M_{\star} [M_{\odot}]$', fontsize=16)

axs[0].legend()
axs[0].set_ylabel(r'$M_{peb}$ [M$_\oplus$]', fontsize=16)

In [ ]:
t1 = 1
t = 0.1
def mstar_scaling(mstar):
    return ((mstar**(-4/7))**3)*mstar
Mstar = np.geomspace(0.1, 10, 100)*u.M_sun.to(u.M_earth)
fig, axs =plt.subplots(1, 1, figsize=(6, 6))
for i, mstar in enumerate(Mstar):
    params = code_gas.Params(H_r_model = 'irradiated', star_mass = mstar, M_dot_star=0)
    axs.scatter(Mstar[i]*u.M_earth.to(u.M_sun), (M_dot_star_t_Mstar(t, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color='gold', label = "t = "+str(t)+" Myr" if i ==0 else None)
    axs.scatter(Mstar[i]*u.M_earth.to(u.M_sun), (M_dot_star_t_Mstar(t1, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color='orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

#axs.loglog(mstar, mstar_scaling(mstar), color='gold')
axs.set_xlabel(r'$M_{\star} [M_{\odot}]$', fontsize=16)
axs.set_yscale('log')
axs.set_xscale('log')
axs.set_ylabel(r'$\dot{M}_{\star} [M_{\odot}/yr]$', fontsize=16)
axs.tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
axs.tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.legend()
plt.savefig('figures/Mdotstar_scaling_1Myr.png', dpi=300, bbox_inches='tight')


In [ ]:
params = code_gas.Params(H_r_model = 'irradiated')
sim_params = code_gas.SimulationParams()
r = np.geomspace(1e-1, 50, 100)
t = 0.2
c = ['gold', 'navy']
ls = ['-', '--']
#fig, axs = plt.subplots(1, 2, figsize=(12, 6))
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
for i, params in enumerate([code_gas.Params(H_r_model = 'irradiated'), code_gas.Params(H_r_model = 'Lambrechts_mixed')]):
    axs[0].loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), color=c[i], label = 'initial mass, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2DB_trans(r, t, params), color='navy',  linestyle = ls[i], label = '3D/2D Bondi accretion new, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2DH_trans(r, t, params), color='green', linestyle = ls[i],  label = '3D/2D Hill accretion new, '+str(params.H_r_model))
    axs[1].loglog(r, M_3D_2D_Bondi_new(r, t, params), color='gold', linestyle = ls[i],  label = '3D/2D Bondi accretion new, '+str(params.H_r_model))


axs[0].axhline(8.314566896275828e-06, color='black', linestyle='--')
axs[0].tick_params(axis='both', which='major', direction = 'in', labelsize=14, size = 10)
axs[0].tick_params(axis='both', which='minor', direction = 'in', labelsize=14, size = 8)
axs[0].set_xlabel(r'$r$ [AU]', fontsize=16)
axs[0].set_ylabel(r'$M_{peb}$ [M$_\oplus$]', fontsize=16)
axs[0].set_title('Initial mass of embros', fontsize=16)
axs[0].legend()
axs[1].legend()


In [ ]:

sim_irr_1 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_irr_10 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json")
sim_params_irr_1 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
sim_params_irr_10 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_1 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_10 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')



params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }
params = code_gas.Params(**params_dict, H_r_model='irradiated', epsilon_el=1, epsilon_heat=1)

def M_3D_2DH_trans_debug_drift_ep(position, t, params):

    stokes_peb = st_drift_epstein(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))

def M_3D_2DH_trans_debug_drift_st(position, t, params):

    stokes_peb = st_drift_stokes(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))
def M_3D_2DH_trans_debug_frag(position, t, params):

    stokes_peb = st_frag(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))


# print(M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_in, params))
fig, axs =plt.subplots(1,1, figsize = (8,8))
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_ep(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift epstein', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_st(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift stokes', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_frag(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st fragt', linestyle = '--') 
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'stokes ', color = 'black')

axs.set_ylim(1e-6, 1e2)
axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_ylabel('$M_{\mathrm{3D/2D}}$ [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plot.all_y_ticks(axs, 100)
axs.legend()
plt.savefig("figures/transition_mass.png", bbox_inches='tight')

### Pebble isolation mass

In [ ]:
r = np.geomspace(4e-3, 1e2, num=1000)
t = 0.3
params = code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)

fig, axs = plt.subplots(1,1, figsize = (8,8))
#axs.loglog(r, peb_iso_liu(r, t, params), label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, Liu et al. 2019",  color = 'black')
#axs.loglog(r, M_peb_iso_Bitsch(r, t, params),label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, Bitsch et al. 2018")

params_list = [code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0), code_gas.Params(H_r_model = 'irradiated', star_mass=0.55*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0), code_gas.Params(H_r_model = 'irradiated', star_mass=1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)]

for params in params_list:
    axs.loglog(r, M_peb_iso(r, t, params),label = str(params.H_r_model)+", $M_{\star}$= "+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")

axs.grid(True, which='both', linestyle='--', alpha=0.5)

axs.set_xlabel("r [AU]", size = 15)
axs.set_ylabel("$M [M_{\oplus}]$", size = 15)
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.legend()
axs.set_title("Pebble isolation mass, $M_{\star} = 0.1 M_{\odot}$", size = 15)
axs.legend()
plt.savefig("figures/pebble_iso.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (8,8))
Mstar = (np.geomspace(0.01,1, num =100)*u.M_sun).to(u.M_earth).value
ice =[]
c = mpl.cm.Blues(np.linspace(0.1, 1, 10))
times = np.linspace(0.1,1, num = 10)

for j,t in enumerate(times):
    for i,m in enumerate(Mstar):
        params = code_gas.Params(H_r_model = 'irradiated', star_mass=m, M_dot_star=0)
        #r_ice = iceline_irr(t, 170, params)
        r_ice = 0.1
        print('r_ice', r_ice)
        ice.append(r_ice)
        axs.scatter((Mstar[i]*u.M_earth).to(u.M_sun), M0_pla_Mstar(r_ice, t, sigma_gas_steady_state(r_ice, t, params), params), color = c[j],label = "t = "+str(t)[:3]+" Myr" if i ==0 else None)
        #axs.scatter((Mstar[i]*u.M_earth).to(u.M_sun), M0_pla_Mstar(r_ice, t1, sigma_gas_steady_state(r_ice, t1, params), params), color = 'orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

plot.all_y_ticks(axs, num_ticks=100)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.set_xlabel("$M_{\star} [M_{\odot}]$", size = 15)
axs.set_ylabel("$M_0 [M_{\oplus}]$", size = 15)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.set_title("Initial mass, irradiated, $a_{p,0}$ = ["+str(ice[0])[:4]+"," +str(ice[-1])[:4]+"] AU", size = 15)
plt.savefig("figures/initial_mass_mstar_irrad_01au.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (8,8))
position = (np.geomspace(0.1, 50, num =100)*u.au).value
t = 0.1
t1=1
ice =[]
for i,r in enumerate(position):
    params = code_gas.Params(H_r_model = 'irradiated', star_mass=0.1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=0)
    axs.scatter(position[i], M0_pla_Mstar(r, t, sigma_gas_steady_state(r, t, params), params), color = 'gold',label = "t = "+str(t)+" Myr" if i ==0 else None)
    print('star mass', params.star_mass*u.M_earth.to(u.M_sun))
    axs.scatter(position[i], M0_pla_Mstar(r, t1, sigma_gas_steady_state(r, t1, params), params), color = 'orange', label = "t = "+str(t1)+" Myr" if i ==0 else None)

plot.all_y_ticks(axs, num_ticks=100)
axs.grid(True, which='both', linestyle='--', alpha=0.5)
axs.set_xlabel("r [AU]", size = 15)
axs.set_ylabel("$M_0 [M_{\oplus}]$", size = 15)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.set_title("Initial mass, irradiated, $M_{\star} = 0.1 M_{\odot}$  ", size = 15)#["+str(ice[0])[:4]+"," +str(ice[-1])[:4]+"] AU", size = 15)
plt.savefig("figures/initial_mass_pos_irrad.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (13,6))
Mstar = (np.geomspace(0.1,1, num =10)*u.M_sun).to(u.M_earth).value
ice =[]
c = mpl.cm.Blues(np.linspace(0.1, 1, 10))
c2 = mpl.cm.Oranges(np.linspace(0.1, 1, 10))

times = np.linspace(0.3, 10, num = 1000)

for i,m in enumerate(Mstar):
    params = code_gas.Params(H_r_model = 'irradiated', star_mass=m, M_dot_star=0)
    params2 = code_gas.Params(H_r_model = 'irradiated', star_mass=m, M_dot_star=None)
    r_ice = 1
    axs[0].loglog(times, (M_dot_star(times, params2)*u.M_earth/u.Myr).to(u.M_sun/u.yr), label = 'Hartmann et. al 2016' if i == 0 else None, color = c2[i])
    axs[0].loglog(times, (M_dot_star(times, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), label = 'Liu et. al 2019, M ='+str(m*u.M_earth.to(u.M_sun))+' $M_{\odot}$' if i == 0 or i == len(Mstar)-1 else None, color = c[i])

import scipy.integrate as spi
def total_gas_mass(params, t_start=0.3, t_end=5.0, other_params=None):
    result, err = spi.quad(M_dot_star, t_start, t_end, args=(params))
    return result*u.M_earth.to(u.M_sun)  # in M_sun
def total_gas_mass_gen(function, params, t_start=0.3, t_end=5.0, other_params=None):
    result, err = spi.quad(function, t_start, t_end, args=(params))
    return result*u.M_earth.to(u.M_sun)  # in M_sun
def linear_scaling(t, params):
    MdotH16 = M_dot_star_t(t)
    return MdotH16*(params.star_mass/(1*const.M_sun.cgs.to(u.M_earth).value))



Mstar = (np.geomspace(0.01,1, num =10)*u.M_sun).to(u.M_earth).value
t_end = 5
gas_mass_liu = [total_gas_mass(code_gas.Params(H_r_model = 'irradiated', star_mass=m,  M_dot_star=0), t_start = 0.3, t_end =t_end) for m in Mstar]
gas_mass_liu05 = [total_gas_mass(code_gas.Params(H_r_model = 'irradiated', star_mass=m,  M_dot_star=0), t_start = 0.5, t_end =t_end) for m in Mstar]
gas_mass_liu1 = [total_gas_mass(code_gas.Params(H_r_model = 'irradiated', star_mass=m,  M_dot_star=0), t_start = 1, t_end =t_end) for m in Mstar]
gas_mass_liu3 = [total_gas_mass(code_gas.Params(H_r_model = 'irradiated', star_mass=m,  M_dot_star=0), t_start = 3, t_end =t_end) for m in Mstar]

gas_mass_hart = [total_gas_mass(code_gas.Params(H_r_model = 'irradiated', star_mass=m,  M_dot_star=None), t_start = 0.3, t_end =t_end) for m in Mstar]
gas_mass_lin = [total_gas_mass_gen(linear_scaling, code_gas.Params(H_r_model = 'irradiated', star_mass=m), t_start = 0.3, t_end =t_end) for m in Mstar]
axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_liu, label = 'gas mass, Liu et. al 2019, $t_{\mathrm{in}}$ = 0.3 Myr', color = 'navy')
axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_lin, label = 'gas mass, linear scaling, $t_{\mathrm{in}}$ = 0.3 Myr', color = 'dodgerblue')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_liu05, label = 'gas mass, Liu et. al 2019, t = 0.5 Myr', color = 'dodgerblue')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_liu1, label = 'gas mass, Liu et. al 2019, t = 1 Myr', color = 'steelblue')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_liu3, label = 'gas mass, Liu et. al 2019, t = 3 Myr', color = 'lightblue')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_hart, label = 'gas mass, Hartmann et. al 2016', color = 'orange')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_liu*np.ones_like(gas_mass_liu)*params.Z, label = 'pebble mass, Liu et. al 2019', color = 'teal')
# axs[1].loglog(Mstar*u.M_earth.to(u.M_sun), gas_mass_hart*np.ones_like(gas_mass_hart)*params.Z, label = 'pebble mass, Hartmann et. al 2016', color = 'gold')
for i in range(2):
    plot.all_y_ticks(axs[0], num_ticks=100)
    axs[i].grid(True, which='both', linestyle='--', alpha=0.5)
    axs[i].legend()
    axs[i].tick_params(axis='both', which = 'major', size = 10, direction='in')
    axs[i].tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs[1].set_ylabel("$M_{\mathrm{disc}} [M_{\odot}]$", size = 15)
axs[1].set_xlabel("$M_{\star} [M_{\odot}]$", size = 15)
axs[0].set_xlabel("t [Myr]", size = 15)
axs[0].set_ylabel("$\dot{M}_{\star} [M_{\odot}/yr]$", size = 15)
axs[0].set_ylim(1e-11, 1e-7)
axs[0].set_title("Gas accretion rate", size = 15)
axs[1].set_title("Total gas mass over [$t_{\mathrm{in}}$-5] Myr", size = 15)

plt.savefig("figures/mdot_disc_mass5Myr.png", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (8,8))
params = code_gas.Params(H_r_model = 'irradiated', star_mass=1*const.M_sun.cgs.to(u.M_earth).value, M_dot_star=None)
axs.loglog(r, M0_pla(r, t, sigma_gas_steady_state(r, t, params), params), label = "old M0_pla")
axs.loglog(r, M0_pla_Mstar(r, t, sigma_gas_steady_state(r, t, params), params), label = "New M0_pla")